In [1]:
from neo4j import GraphDatabase
import sqlite3


uri = "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "part2"))
session = driver.session()

In [2]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = str(row[idx])
    return d

def create_node(session, params):
    session.run("CREATE(p:Person $param)", param = params)



In [3]:
conn = sqlite3.connect("../data/ucheckra_.db")
conn.row_factory = dict_factory
checkra_follows = conn.cursor().execute("select * from following").fetchall()

In [59]:
# add original follows
for d in checkra_follows[:10]:
    create_node(session, d)



In [149]:
def create_connection(session, to_id, from_id):
    result = session.run(f"match (n:Person {from_id})")
                        # "merge (n)-[:Follows]- (m)",
                        # "return n.username",
                        # from_id = from_id)
    # else:
    #     result = session.run("match (n:Person {username: $from_username}), (m:Person {id: $to_id})"
    #                         # "merge (n)-[:Follows]- (m)",
    #                         "return n.username, m.username",
    #                         from_username = from_username, to_id = to_id) 

    # print(result.data()[0])
# create_node(session, {'id': '946156845026115584', 'name': 'Muaz', 'username': 'MuazCoin'})
create_connection(session, str(946156845026115584), {id: "Ninjascalp"})

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '<': expected whitespace, a property key name, '}', an identifier or UnsignedDecimalInteger (line 1, column 18 (offset: 17))
"match (n:Person {<built-in function id>: 'Ninjascalp'})"
                  ^}

In [175]:
bruh = lambda param: list(param.keys())[0]
# "{"+"".join([p for p in list(param.keys())]) + "}"
d = {"id": "Ninjascalp"}
bruh(d)
# TODO trying to make methods work for either id or username, need to finsih adding relationships for all people I follow
# print(list(d.keys())[0])

'id'

In [143]:
str({"id": "Ninjascalp"})

"{'id': 'Ninjascalp'}"

In [126]:
def check_connection(session, from_id, to_id):
    result = session.run("MATCH (n:Person {id: $from_id})-[r]- (m:Person {id: $to_id})"
                         "RETURN n,type(r),m",
                         from_id = from_id, to_id = to_id)
    try:
        result.peek()
    except ResultError:
        yield False
    else:
        record = [i for i in result]
        yield True
        yield record[0]["n"]["name"], record[0]["type(r)"], record[0]["m"]["name"]

gen = check_connection(session, str(943958987556507653), str(1962290894))
if next(gen):
    print(next(gen))
else:
    print("not a relation")

('Ninja', 'Follows', 'cevo')


In [78]:
def user_edges(session, username):
    conn_temp = sqlite3.connect(f"../data/users/u{username}.db")
    conn_temp.row_factory = dict_factory
    temp_follows = conn_temp.cursor().execute("select * from following").fetchall()
    for d in temp_follows:
        create_node(session, d)
        create_connection(session, s)
        
user_edges(session, "Ninjascalp")

{'id': '1962290894', 'name': 'cevo', 'username': 'cryptocevo'}
{'id': '946156845026115584', 'name': 'Muaz', 'username': 'MuazCoin'}
{'id': '1184226678870028289', 'name': 'ExoCharts.com', 'username': 'ExochartsC'}
{'id': '1073132650309726208', 'name': 'CL', 'username': 'CL207'}
{'id': '1039833297751302144', 'name': 'Whale Alert', 'username': 'whale_alert'}
{'id': '1364691158643965953', 'name': 'pepe', 'username': 'PepeXBT'}
{'id': '1181494351031083008', 'name': 'Joe007 signals·alerts·funds are scams, all of them', 'username': 'J0E007'}
{'id': '1368261032066621588', 'name': '𝕀𝕟𝕊𝕚𝕝𝕚𝕔𝕠', 'username': 'insiliconot'}
{'id': '535241898', 'name': 'whisper trader', 'username': 'cryptoWhisper'}
{'id': '827237880300974081', 'name': 'BitBit.FullSend', 'username': 'BitBitCrypto'}
{'id': '1566114613', 'name': 'SalsaTekila', 'username': 'SalsaTekila'}
{'id': '1650876864', 'name': 'Trizzy', 'username': 'ImTrizzy'}
{'id': '982257333278457858', 'name': 'Gone Broke', 'username': 'gonebroke_'}
{'id': '2603

In [9]:
conn.cursor().execute("select * from following where username = 'Ninjascalp'").fetchall()

[{'id': 943958987556507653, 'name': 'Ninja', 'username': 'Ninjascalp'}]